# Process All the Files


pip install opencv-python


# Stereo to depthmap conversion

In [ ]:
import numpy as np
import cv2
import argparse
import sys

def depth_map(imgL, imgR):
    """ Depth map calculation. Works with SGBM and WLS. Need rectified images, returns depth map (left to right disparity) """
    window_size = 10  # Window size for SGBM
    min_disp = 0
    num_disp = 16 * 5  # Needs to be divisible by 16

    left_matcher = cv2.StereoSGBM_create(
        # minDisparity=min_disp,
        numDisparities=16,
        blockSize=window_size,
        P1=8 * 3 * window_size ** 2,
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    lmbda = 80000
    sigma = 1.5

    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)

    displ = left_matcher.compute(imgL, imgR).astype(np.int16)
    dispr = right_matcher.compute(imgR, imgL).astype(np.int16)
    filtered_img = wls_filter.filter(displ, imgL, None, dispr)

    # Normalize the filtered disparity map for visualization
    filtered_img = cv2.normalize(filtered_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    filtered_img = np.uint8(filtered_img)

    return filtered_img

def parse_matrix_from_file(file_path, matrix_name):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith(matrix_name):
                values = list(map(float, line.split(':')[1].strip().split()))
                return np.array(values).reshape((3, 4) if 'P' in matrix_name else (3, 3))
    raise ValueError(f"Matrix {matrix_name} not found in file {file_path}")

def parse_translation_vector(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('Tr_velo_to_cam'):
                values = list(map(float, line.split(':')[1].strip().split()))
                return np.array(values).reshape((3, 4))
    raise ValueError(f"Translation vector Tr_velo_to_cam not found in file {file_path}")

def disparity_to_depth(disparity, focal_length, baseline):
    """Convert disparity map to depth map."""
    with np.errstate(divide='ignore'):
        depth = (focal_length * baseline) / disparity
    depth[disparity == 0] = 0  # Mask out disparity values of 0 (infinite depth)
    return depth

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Stereo depth map')
    parser.add_argument('--calibration_file', type=str, required=True, help='Path to the calibration file')
    parser.add_argument('--left_image', type=str, required=True, help='Path to the left image')
    parser.add_argument('--right_image', type=str, required=True, help='Path to the right image')

    args = parser.parse_args()

    # Parse matrices from the calibration file
    P2 = parse_matrix_from_file(args.calibration_file, 'P2')
    P3 = parse_matrix_from_file(args.calibration_file, 'P3')
    R0_rect = parse_matrix_from_file(args.calibration_file, 'R0_rect')
    T = parse_translation_vector(args.calibration_file)

    K1 = P2[:, :3]
    K2 = P3[:, :3]
    R1 = R0_rect
    R2 = R0_rect
    P1 = P2
    P2 = P3

    # Extract focal length and baseline
    focal_length = K1[0, 0]  # Assuming fx is the same for both cameras
    baseline = np.linalg.norm(T[:, 3])  # Baseline is the norm of the translation vector

    # Read the images
    leftFrame = cv2.imread(args.left_image)
    rightFrame = cv2.imread(args.right_image)

    if leftFrame is None or rightFrame is None:
        print("Can't open the images!")
        sys.exit(-9)

    height, width, _ = leftFrame.shape

    # Generate rectification maps
    leftMapX, leftMapY = cv2.initUndistortRectifyMap(K1, None, R1, P1, (width, height), cv2.CV_32FC1)
    rightMapX, rightMapY = cv2.initUndistortRectifyMap(K2, None, R2, P2, (width, height), cv2.CV_32FC1)

    left_rectified = cv2.remap(leftFrame, leftMapX, leftMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    right_rectified = cv2.remap(rightFrame, rightMapX, rightMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)

    gray_left = cv2.cvtColor(left_rectified, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_rectified, cv2.COLOR_BGR2GRAY)

    disparity_image = depth_map(gray_left, gray_right)

    # Convert disparity map to depth map
    depth_image = disparity_to_depth(disparity_image, focal_length, baseline)

    cv2.imshow('left_rectified', left_rectified)
    cv2.imshow('right_rectified', right_rectified)
    cv2.imshow('Disparity', disparity_image)
    cv2.imshow('Depth', depth_image)

    cv2.waitKey(0)  # Wait for any key to be pressed
    cv2.destroyAllWindows()


## Stereo to Disparity map

In [1]:
import numpy as np
import cv2
import argparse
import sys

def depth_map(imgL, imgR):
    """ Depth map calculation. Works with SGBM and WLS. Need rectified images, returns depth map (left to right disparity) """
    window_size = 5  # Window size for SGBM
    min_disp = 0
    num_disp = 16 * 5  # Needs to be divisible by 16

    left_matcher = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=window_size,
        P1=8 * 3 * window_size ** 2,
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

    lmbda = 80000
    sigma = 1.5

    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)

    displ = left_matcher.compute(imgL, imgR).astype(np.int16)
    dispr = right_matcher.compute(imgR, imgL).astype(np.int16)
    filtered_img = wls_filter.filter(displ, imgL, None, dispr)

    # Normalize the filtered disparity map for visualization
    filtered_img = cv2.normalize(filtered_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    filtered_img = np.uint8(filtered_img)

    return filtered_img

def parse_matrix_from_file(file_path, matrix_name):
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith(matrix_name):
                values = list(map(float, line.split(':')[1].strip().split()))
                return np.array(values).reshape((3, 4) if 'P' in matrix_name else (3, 3))
    raise ValueError(f"Matrix {matrix_name} not found in file {file_path}")

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Stereo depth map')
    parser.add_argument('--calibration_file', type=str, required=True, help='Path to the calibration file')
    parser.add_argument('--left_image', type=str, required=True, help='Path to the left image')
    parser.add_argument('--right_image', type=str, required=True, help='Path to the right image')

    args = parser.parse_args()

    # Parse matrices from the calibration file
    P2 = parse_matrix_from_file(args.calibration_file, 'P2')
    P3 = parse_matrix_from_file(args.calibration_file, 'P3')
    R0_rect = parse_matrix_from_file(args.calibration_file, 'R0_rect')

    K1 = P2[:, :3]
    K2 = P3[:, :3]
    R1 = R0_rect
    R2 = R0_rect
    P1 = P2
    P2 = P3

    # Read the images
    leftFrame = cv2.imread(args.left_image)
    rightFrame = cv2.imread(args.right_image)

    if leftFrame is None or rightFrame is None:
        print("Can't open the images!")
        sys.exit(-9)

    height, width, _ = leftFrame.shape

    # Generate rectification maps
    leftMapX, leftMapY = cv2.initUndistortRectifyMap(K1, None, R1, P1, (width, height), cv2.CV_32FC1)
    rightMapX, rightMapY = cv2.initUndistortRectifyMap(K2, None, R2, P2, (width, height), cv2.CV_32FC1)

    left_rectified = cv2.remap(leftFrame, leftMapX, leftMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    right_rectified = cv2.remap(rightFrame, rightMapX, rightMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)

    gray_left = cv2.cvtColor(left_rectified, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_rectified, cv2.COLOR_BGR2GRAY)

    disparity_image = depth_map(gray_left, gray_right)

    cv2.imshow('left_rectified', left_rectified)
    cv2.imshow('right_rectified', right_rectified)
    cv2.imshow('Disparity', disparity_image)

    cv2.waitKey(0)  # Wait for any key to be pressed
    cv2.destroyAllWindows()


usage: ipykernel_launcher.py [-h] --calibration_file CALIBRATION_FILE
                             --left_image LEFT_IMAGE --right_image RIGHT_IMAGE
ipykernel_launcher.py: error: the following arguments are required: --calibration_file, --left_image, --right_image


SystemExit: 2

/Users/adarshjohny/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import cv2
import numpy as np
import os

# Load camera calibration data
def load_calib(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        P0 = np.array([float(x) for x in lines[0].split(':')[1].strip().split()]).reshape(3, 4)
        P1 = np.array([float(x) for x in lines[1].split(':')[1].strip().split()]).reshape(3, 4)
        
        # Pad P0 with a bottom row of [0, 0, 0, 1] to make it 4x4
        P0 = np.vstack((P0, np.array([0, 0, 0, 1])))
    return P0, P1

def process_images(left_folder, right_folder, output_folder, calib_folder):
    # List all files in the left folder
    left_files = os.listdir(left_folder)

    for left_file in left_files:
        if left_file.endswith('.png'):
            # Form corresponding paths for left and right images and calibration data
            left_image_path = os.path.join(left_folder, left_file)
            right_image_path = os.path.join(right_folder, left_file)
            calib_file = os.path.join(calib_folder, left_file.replace('.png', '.txt'))

            # Load camera calibration data
            P0, P1 = load_calib(calib_file)

            # Load left and right images
            left_img = cv2.imread(left_image_path)
            right_img = cv2.imread(right_image_path)

            # Create a StereoSGBM object
            stereo = cv2.StereoSGBM_create(numDisparities=48, blockSize=1)

            # Compute the disparity map
            disparity = stereo.compute(left_img, right_img)

            # Reproject the disparity map to 3D space to obtain the depth map
            depth_map = cv2.reprojectImageTo3D(disparity, P0)

            # Save the depth map as a PNG file with the same name in the output folder
            output_file = os.path.join(output_folder, left_file.replace('.png', '_depth.png'))
            print("Saving depth map to:", output_file)
            success = cv2.imwrite(output_file, depth_map)
            if success:
                print("Depth map saved successfully.")
            else:
                print("Error: Unable to save depth map.")

# Paths for training and test datasets
train_left_folder = 'Project_Files/left/training'
train_right_folder = 'Project_Files/right/training'
train_output_folder = 'Project_Files/depth_images_local/training'
train_calib_folder = 'Project_Files/calib/training'

test_left_folder = 'Project_Files/left/testing'
test_right_folder = 'Project_Files/right/testing'
test_output_folder = 'Project_Files/depth_images_local/testing'
test_calib_folder = 'Project_Files/calib/testing'

# Process training images
process_images(train_left_folder, train_right_folder, train_output_folder, train_calib_folder)

# Process test images
process_images(test_left_folder, test_right_folder, test_output_folder, test_calib_folder)


Saving depth map to: Project_Files/depth_images_local/training/000792_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000001_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000000_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000002_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000003_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000792_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000001_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000000_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000002_depth.png
Depth map saved successfully.
Saving depth m